In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
Setup complete ✅ (96 CPUs, 334.6 GB RAM, 6410.9/8062.4 GB disk)


In [2]:
!pip uninstall -y wandb

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1BTvp0FPVcivYNLjzxn1")
project = rf.workspace("object-detection-sn8ac").project("helmate-gxxio-emckf")
version = project.version(1)
dataset = version.download("yolov8")
                

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 33.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
loading Robo

Extracting Dataset Version Zip to helmate-1 in yolov8:: 100%|██████████| 6118/6118 [00:01<00:00, 5771.22it/s]


In [ ]:
# Train YOLOv8n on COCO8 for 50 epochs
!yolo train model=yolov8s.pt data="/kaggle/working/helmate-1/data.yaml" epochs=50 imgsz=640 batch=16 device=0,1

Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/helmate-1/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

In [5]:
pip install ultralytics opencv-python numpy


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

class HelmetDetectionSystem:
    def __init__(self, model_path, class_names):
        """
        Initialize the helmet detection system with YOLO.
        
        Args:
            model_path: Path to the trained YOLO model (.pt file)
            class_names: List of class names in order [helmet, license_plate, motorcyclist]
        """
        self.model = YOLO(model_path)
        self.class_names = class_names
        self.motorcyclist_index = class_names.index('motorcyclist')
        self.helmet_index = class_names.index('helmet')
        
        # Detection colors
        self.colors = {
            'motorcyclist': (0, 255, 0),  # Green
            'helmet': (0, 255, 0),       # Green
            'no_helmet': (0, 0, 255),     # Red
            'license_plate': (255, 0, 0)  # Blue
        }
    
    def detect_objects(self, frame, confidence_threshold=0.5):
        """Detect objects in a frame using YOLO."""
        results = self.model(frame, conf=confidence_threshold)
        detections = []
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls)
                confidence = float(box.conf)
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                class_name = self.class_names[class_id]
                box_coords = (x1, y1, x2-x1, y2-y1)
                detections.append((class_name, confidence, box_coords))
        return detections
    
    def process_frame(self, frame):
        """Process a single frame with vertical orientation."""
        # Rotate frame to vertical if width > height
        if frame.shape[1] > frame.shape[0]:  # If width > height
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            
        detections = self.detect_objects(frame)
        motorcyclists = [d for d in detections if d[0] == 'motorcyclist']
        
        for motorcyclist in motorcyclists:
            _, _, mc_box = motorcyclist
            x, y, w, h = mc_box
            cv2.rectangle(frame, (int(x), int(y)), (int(x+w), int(y+h)), 
                         self.colors['motorcyclist'], 2)
            
            has_helmet = False
            for detection in detections:
                class_name, _, h_box = detection
                if class_name == 'helmet' and self._check_overlap(mc_box, h_box):
                    has_helmet = True
                    hx, hy, hw, hh = h_box
                    cv2.rectangle(frame, (int(hx), int(hy)), (int(hx+hw), int(hy+hh)), 
                                 self.colors['helmet'], 2)
                    break
            
            label = "Helmet" if has_helmet else "No Helmet"
            color = self.colors['helmet'] if has_helmet else self.colors['no_helmet']
            cv2.putText(frame, label, (int(x), int(y-10)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        for detection in detections:
            if detection[0] == 'license_plate':
                x, y, w, h = detection[2]
                cv2.rectangle(frame, (int(x), int(y)), (int(x+w), int(y+h)), 
                             self.colors['license_plate'], 2)
                cv2.putText(frame, "License Plate", (int(x), int(y-10)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                           self.colors['license_plate'], 2)
        
        return frame
    
    def _check_overlap(self, box1, box2, overlap_threshold=0.5):
        """Check if two bounding boxes overlap significantly."""
        x_left = max(box1[0], box2[0])
        y_top = max(box1[1], box2[1])
        x_right = min(box1[0] + box1[2], box2[0] + box2[2])
        y_bottom = min(box1[1] + box1[3], box2[1] + box2[3])
        
        if x_right < x_left or y_bottom < y_top:
            return False
            
        intersection_area = (x_right - x_left) * (y_bottom - y_top)
        box1_area = box1[2] * box1[3]
        box2_area = box2[2] * box2[3]
        overlap_ratio = intersection_area / min(box1_area, box2_area)
        
        return overlap_ratio >= overlap_threshold
    
    def process_video(self, video_path, output_path, fps=None):
        """Process video with guaranteed vertical output."""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return
            
        # Get original video properties
        orig_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        orig_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Force vertical output dimensions (height > width)
        output_width = min(orig_width, orig_height)
        output_height = max(orig_width, orig_height)
        
        # Set output FPS
        if fps is None:
            fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Video writer with vertical orientation
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (output_width, output_height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Process frame (automatically rotates to vertical)
            processed_frame = self.process_frame(frame)
            
            # Ensure output matches desired dimensions
            if processed_frame.shape[1] != output_width or processed_frame.shape[0] != output_height:
                processed_frame = cv2.resize(processed_frame, (output_width, output_height))
            
            out.write(processed_frame)
            cv2.imshow('Helmet Detection', processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"Vertical video saved to {output_path}")


if __name__ == "__main__":
    # Initialize with your paths
    detector = HelmetDetectionSystem(
        model_path=r'C:\Users\Daud\Downloads\traffic_safety_yolov8\best.pt',
        class_names=['helmet', 'license_plate', 'motorcyclist']
    )
    
    # Process video with vertical output
    detector.process_video(
        video_path=r'C:\Users\Daud\Downloads\traffic_safety_yolov8\5 s.mp4',
        output_path=r'C:\Users\Daud\Downloads\traffic_safety_yolov8\1output_vertical.mp4'
    )


0: 640x384 1 motorcyclist, 230.5ms
Speed: 5.2ms preprocess, 230.5ms inference, 7.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 motorcyclists, 148.7ms
Speed: 3.6ms preprocess, 148.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 196.0ms
Speed: 3.1ms preprocess, 196.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 motorcyclists, 207.1ms
Speed: 2.7ms preprocess, 207.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 motorcyclists, 204.1ms
Speed: 1.9ms preprocess, 204.1ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 motorcyclist, 251.0ms
Speed: 2.6ms preprocess, 251.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 motorcyclists, 197.7ms
Speed: 1.9ms preprocess, 197.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 motorcyclists, 200.5ms
Speed: 3.1ms preproc